# Simple Entry Removal Algorithm 1 (SER 1)

In [1]:
import numpy as np
import datetime
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [2]:
def add_up_a_decomposition_float(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=float)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# The SER 1 algorithm

In [3]:
# This function is used in the SER 1 function.
# residue_matrix_R is a np.ndarray.
# PBN_row_num is the number of rows present in residue_matrix_R.
# PBN_col_num is the number of columns present in residue_matrix_R.
def find_smallest_nonzero_entry(residue_matrix_R, PBN_row_num, PBN_col_num):
    row_counter = 0
    col_counter = 0
    while residue_matrix_R[row_counter, col_counter] == 0:
        row_counter += 1
    smallest_entry = residue_matrix_R[row_counter, col_counter]
    smallest_entry_row_pos = row_counter
    smallest_entry_col_pos = col_counter

    del row_counter
    del col_counter
    for col_counter in range(PBN_col_num):
        for row_counter in range(PBN_row_num):
            current_entry = residue_matrix_R[row_counter, col_counter]
            if (current_entry > 0) and (current_entry < smallest_entry):
                smallest_entry = current_entry
                smallest_entry_row_pos = row_counter
                smallest_entry_col_pos = col_counter

    return smallest_entry, smallest_entry_row_pos, smallest_entry_col_pos

In [6]:
def float_SER_1(PBN_matrix_P, stopping_error):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                          PBN_row_num, PBN_col_num)
    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
#    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (np.linalg.norm(residue_matrix_R, ord='fro') >= stopping_error):
        k += 1
        chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                              PBN_row_num, PBN_col_num)
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai, np.linalg.norm(residue_matrix_R, ord='fro')

# Floating-Point Matrix Experiment

In [5]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## $8 \times 8$ TPMs

In [26]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './8-by-8/dim8_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_1(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_1)/dim8_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_1)/dim8_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 00:31:53.878882

iteration 1
Time now is: 2025-04-12 00:31:53.935136

iteration 2
Time now is: 2025-04-12 00:31:53.951515

iteration 3
Time now is: 2025-04-12 00:31:53.970543

iteration 4
Time now is: 2025-04-12 00:31:53.986693

iteration 5
Time now is: 2025-04-12 00:31:54.003432

iteration 6
Time now is: 2025-04-12 00:31:54.067093

iteration 7
Time now is: 2025-04-12 00:31:54.096987

iteration 8
Time now is: 2025-04-12 00:31:54.120564

iteration 9
Time now is: 2025-04-12 00:31:54.138903



## $16 \times 16$ TPMs

In [40]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './16-by-16/dim16_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_1(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_1)/dim16_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_1)/dim16_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 02:00:59.802170

iteration 1
Time now is: 2025-04-12 02:00:59.816574

iteration 2
Time now is: 2025-04-12 02:00:59.829814

iteration 3
Time now is: 2025-04-12 02:00:59.841052

iteration 4
Time now is: 2025-04-12 02:00:59.854138

iteration 5
Time now is: 2025-04-12 02:00:59.866700

iteration 6
Time now is: 2025-04-12 02:00:59.879360

iteration 7
Time now is: 2025-04-12 02:00:59.892911

iteration 8
Time now is: 2025-04-12 02:00:59.904808

iteration 9
Time now is: 2025-04-12 02:00:59.918144



## $32 \times 32$ TPMs

In [39]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './32-by-32/dim32_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_1(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_1)/dim32_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_1)/dim32_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 02:00:41.618169

iteration 1
Time now is: 2025-04-12 02:00:41.727177

iteration 2
Time now is: 2025-04-12 02:00:41.804122

iteration 3
Time now is: 2025-04-12 02:00:41.886761

iteration 4
Time now is: 2025-04-12 02:00:41.973223

iteration 5
Time now is: 2025-04-12 02:00:42.051842

iteration 6
Time now is: 2025-04-12 02:00:42.149234

iteration 7
Time now is: 2025-04-12 02:00:42.251306

iteration 8
Time now is: 2025-04-12 02:00:42.373394

iteration 9
Time now is: 2025-04-12 02:00:42.473124



## $64 \times 64$ TPMs

In [37]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './64-by-64/dim64_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_1(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_1)/dim64_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_1)/dim64_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 01:59:40.455165

iteration 1
Time now is: 2025-04-12 01:59:41.512614

iteration 2
Time now is: 2025-04-12 01:59:42.370963

iteration 3
Time now is: 2025-04-12 01:59:43.294679

iteration 4
Time now is: 2025-04-12 01:59:44.303849

iteration 5
Time now is: 2025-04-12 01:59:45.331243

iteration 6
Time now is: 2025-04-12 01:59:46.318268

iteration 7
Time now is: 2025-04-12 01:59:47.247721

iteration 8
Time now is: 2025-04-12 01:59:48.176914

iteration 9
Time now is: 2025-04-12 01:59:49.047225



# Some random test code

In [18]:
test_matrix = np.random.exponential(size=(8, 8))

for k in range(8):
    test_matrix[:, k] = test_matrix[:, k] / np.sum(test_matrix[:, k])

In [19]:
np.sum(test_matrix, axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
K, x_list, BN_list, error = float_SER_1(test_matrix, 0.001)

In [25]:
K

56

In [22]:
error

0.0008336525765043137

In [23]:
test_matrix

array([[6.76279293e-02, 1.97599922e-01, 1.47272699e-01, 4.79495014e-02,
        1.30179052e-01, 9.03982345e-03, 2.48942344e-03, 1.69069237e-01],
       [7.63401109e-02, 3.28806017e-02, 1.69951894e-02, 5.58888553e-02,
        4.69089837e-02, 3.01244879e-01, 2.53810702e-01, 5.16819806e-02],
       [1.59343918e-01, 9.00666432e-02, 6.10308813e-02, 3.56674583e-02,
        2.04372414e-02, 3.01215020e-01, 7.99901204e-02, 1.19628760e-01],
       [1.92271199e-01, 1.40574756e-01, 1.50380573e-02, 3.51010986e-02,
        2.82038675e-01, 1.36699696e-01, 1.14646021e-01, 7.61854206e-02],
       [1.46546655e-01, 2.60557639e-02, 1.56492980e-01, 6.29101536e-02,
        9.88093154e-02, 1.37980312e-02, 2.27738289e-01, 4.31520494e-01],
       [2.96081677e-02, 4.28946314e-02, 7.29423278e-03, 1.68803484e-02,
        4.92513947e-04, 1.61903647e-01, 1.00704333e-02, 6.75850213e-02],
       [3.15307692e-01, 3.70446007e-01, 2.18452057e-01, 6.20835279e-01,
        1.26433018e-01, 2.73612664e-02, 1.58065805e-01, 9.

In [24]:
add_up_a_decomposition_float(x_list, BN_list, 8, 8)

array([[6.73331886e-02, 1.97305181e-01, 1.47272699e-01, 4.79495014e-02,
        1.30179052e-01, 9.03982345e-03, 2.48942344e-03, 1.69069237e-01],
       [7.63401109e-02, 3.28806017e-02, 1.69951894e-02, 5.58888553e-02,
        4.69089837e-02, 3.01244879e-01, 2.53810702e-01, 5.16819806e-02],
       [1.59343918e-01, 9.00666432e-02, 6.10308813e-02, 3.56674583e-02,
        2.04372414e-02, 3.00920280e-01, 7.99901204e-02, 1.19628760e-01],
       [1.92271199e-01, 1.40574756e-01, 1.50380573e-02, 3.51010986e-02,
        2.82038675e-01, 1.36699696e-01, 1.14646021e-01, 7.61854206e-02],
       [1.46546655e-01, 2.60557639e-02, 1.56492980e-01, 6.29101536e-02,
        9.88093154e-02, 1.37980312e-02, 2.27443548e-01, 4.31225753e-01],
       [2.96081677e-02, 4.28946314e-02, 7.29423278e-03, 1.68803484e-02,
        4.92513947e-04, 1.61903647e-01, 1.00704333e-02, 6.75850213e-02],
       [3.15307692e-01, 3.70446007e-01, 2.18157316e-01, 6.20540538e-01,
        1.26433018e-01, 2.73612664e-02, 1.58065805e-01, 9.